In [ ]:
import torch
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from diffusers import StableDiffusionPipeline, DDIMScheduler
from PIL import Image
import numpy as np
import os
from tqdm.notebook import tqdm
from transformers import CLIPImageProcessor
import json

# Parameters
model_id = "runwayml/stable-diffusion-v1-5"
image_paths = [
    "../data/results/ma-boston_200250_fake_B.png",
    "../data/results/nc-charlotte_200250_fake_B.png",
    "../data/results/ny-manhattan_200250_fake_B.png",
    "../data/results/pa-pittsburgh_200250_fake_B.png"
]
inference_steps = 15000  # increase for better image quality
identifier = f"stable-diffusion-weighted-interpolation_{inference_steps}-inference"  # Unique identifier for this run
output_dir = os.path.join("diffusion-output", identifier)
output_image_size = (512, 512)  # width and height of output image
max_image_dimension = 1024  # Maximum dimension for resizing images

# Example softmax weights (these should sum to 1)
weights = [0.25, 0.35, 0.15, 0.25]  # Replace with your actual weights

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Set device
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

def load_and_preprocess_image(image_path):
    """Load and preprocess an image."""
    # Open the image and convert to RGB
    image = Image.open(image_path).convert("RGB")
    
    # Get the original size
    original_size = image.size
    
    # Calculate the aspect ratio
    aspect_ratio = original_size[0] / original_size[1]
    
    # Determine the new size (ensuring it's divisible by 8 for the VAE)
    if aspect_ratio > 1:
        new_width = min(original_size[0], max_image_dimension)
        new_width = new_width - (new_width % 8)
        new_height = int(new_width / aspect_ratio)
        new_height = new_height - (new_height % 8)
    else:
        new_height = min(original_size[1], max_image_dimension)
        new_height = new_height - (new_height % 8)
        new_width = int(new_height * aspect_ratio)
        new_width = new_width - (new_width % 8)
    
    new_size = (new_width, new_height)
    
    # Create the transform
    transform = Compose([
        Resize(new_size),
        ToTensor(),
        Normalize([0.5], [0.5])
    ])
    
    # Apply the transform
    image_tensor = transform(image).unsqueeze(0).to(device).to(torch.float16)  # Ensure consistent data type
    
    return image_tensor

try:
    # Load the pre-trained Stable Diffusion model with DDIM scheduler
    with tqdm(total=1, desc="Loading Stable Diffusion model") as pbar:
        scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
        pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)  # Use mixed precision
        pipe = pipe.to(device)
        if device.type == "cuda":
            pipe.enable_attention_slicing()
        pbar.update(1)

    # Load and preprocess images
    images = []
    sizes = []
    original_sizes = []
    with tqdm(total=len(image_paths), desc="Loading and preprocessing images") as pbar:
        for image_path in image_paths:
            image = load_and_preprocess_image(image_path)
            images.append(image)
            pbar.update(1)

    print(f"Processed sizes: {sizes}")
    print(f"Original sizes: {original_sizes}")

    # Encode images to latents
    latents = []
    with tqdm(total=len(images), desc="Encoding images to latent representations") as pbar:
        with torch.no_grad():
            for image in images:
                latent = pipe.vae.encode(image).latent_dist.sample() * 0.18215
                latents.append(latent)
                pbar.update(1)

    print(f"Shapes of latents: {[latent.shape for latent in latents]}")

    # Perform weighted interpolation between latents
    print("Performing weighted interpolation...")
    weighted_latent = sum(w * latent for w, latent in zip(weights, latents))

    with torch.no_grad():
        # Scale the latent
        weighted_latent = 1 / 0.18215 * weighted_latent
        # Decode the weighted latent
        decoded_image = pipe.vae.decode(weighted_latent).sample
        # Normalize the decoded image
        decoded_image = (decoded_image / 2 + 0.5).clamp(0, 1)
        # Convert to CPU and then to numpy array
        decoded_image = decoded_image.cpu().permute(0, 2, 3, 1).numpy()[0]

    # Convert to PIL Image
    decoded_image_pil = Image.fromarray((decoded_image * 255).astype(np.uint8))
    # Resize the output image to the desired output size
    decoded_image_pil = decoded_image_pil.resize(output_image_size, Image.LANCZOS)

    # Save the weighted interpolated image
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "weighted_interpolated_image.png")
    decoded_image_pil.save(output_path, quality=95)

    print(f"Weighted interpolation image saved at: {output_path}")

except Exception as e:
    print(f"An error occurred: {e}")